In [1]:

# ================================
# 1) Install and Imports (Colab)
# ================================
# !pip -q install --upgrade transformers datasets peft accelerate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 145.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 53.4 MB/s eta 0:00:00


In [1]:

!pip -q install "transformers==4.44.2" "datasets==2.21.0" "peft==0.10.0" "accelerate==0.34.2" "scikit-learn==1.5.2"

import torch, transformers, peft, datasets, sklearn
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)
print


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 122.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is 

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Torch: 2.9.0+cu126
Transformers: 4.44.2
PEFT: 0.10.0


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

# Load the dataset from Google Drive into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PhD_Thesis_Experiments/GitHub_ToChair/FilteredDataSetForExp_002.csv')

In [7]:

# ================================
# 1) Install compatible packages
# ================================
# !pip -q install "transformers==4.44.2" "datasets==2.21.0" "peft==0.10.0" "accelerate==0.34.2" "scikit-learn==1.5.2"

import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

from peft import LoraConfig, get_peft_model, TaskType

# For reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# ==========================================
# 2) Make sure df is present and normalized
# ==========================================
# df must have columns: consumer_complaint_narrative, hierarchical_label

assert 'consumer_complaint_narrative' in df.columns and 'hierarchical_label' in df.columns, \
    "DataFrame must have columns: consumer_complaint_narrative, hierarchical_label"

df = df[['consumer_complaint_narrative', 'hierarchical_label']].copy()
df.dropna(subset=['consumer_complaint_narrative', 'hierarchical_label'], inplace=True)
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].astype(str).str.strip()
df['hierarchical_label'] = df['hierarchical_label'].astype(str).str.strip()

# Ensure labels are "product::sub-product"; fill missing sub-product with 'None'
def normalize_hier_label(label: str):
    label = label.strip()
    if '::' in label:
        p = label.split('::', 1)
        product = p[0].strip()
        sub = p[1].strip() if p[1].strip() else 'None'
        return f"{product}::{sub}"
    else:
        return f"{label}::None"

df['hierarchical_label'] = df['hierarchical_label'].apply(normalize_hier_label)

# ==========================================================
# 3) Your Hierarchical Metric Functions (exactly as provided)
# ==========================================================
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Function to split hierarchical labels into product and sub-product
def split_hierarchical_label(label):
    if '::' in label:
        return label.split('::')
    else:
        return [label, 'None'] # Handle cases with no sub-product

# Function to calculate hierarchical metrics
def hierarchical_metrics(y_true, y_pred):
    product_true = [split_hierarchical_label(label)[0] for label in y_true]
    sub_product_true = [split_hierarchical_label(label)[1] for label in y_true]
    product_pred = [split_hierarchical_label(label)[0] for label in y_pred]
    sub_product_pred = [split_hierarchical_label(label)[1] for label in y_pred]

    # Calculate metrics at the product level
    product_precision = precision_score(product_true, product_pred, average='weighted', zero_division=0)
    product_recall = recall_score(product_true, product_pred, average='weighted', zero_division=0)
    product_f1 = f1_score(product_true, product_pred, average='weighted', zero_division=0)

    # Calculate metrics at the sub-product level (only for non-None sub-products)
    valid_sub_product_true = [sub for i, sub in enumerate(sub_product_true) if sub != 'None' and sub_product_pred[i] != 'None']
    valid_sub_product_pred = [sub for i, sub in enumerate(sub_product_pred) if sub != 'None' and sub_product_true[i] != 'None']

    sub_product_precision = precision_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_recall = recall_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_f1 = f1_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0

    # Average product and sub-product metrics
    hierarchical_precision = (product_precision + sub_product_precision) / 2
    hierarchical_recall = (product_recall + sub_product_recall) / 2
    hierarchical_f1 = (product_f1 + sub_product_f1) / 2

    return {
        'product_precision': product_precision,
        'product_recall': product_recall,
        'product_f1': product_f1,
        'sub_product_precision': sub_product_precision,
        'sub_product_recall': sub_product_recall,
        'sub_product_f1': sub_product_f1,
        'hierarchical_precision': hierarchical_precision,
        'hierarchical_recall': hierarchical_recall,
        'hierarchical_f1': hierarchical_f1
    }

# ==========================================================
# 4) Build label space and datasets
# ==========================================================
labels = sorted(df['hierarchical_label'].unique())
label2id = {lbl: i for i, lbl in enumerate(labels)}
id2label = {i: lbl for lbl, i in label2id.items()}
num_labels = len(labels)

print(f"Num classes (hierarchical_label): {num_labels}")
print("Sample classes:", labels[:10])

df['label_id'] = df['hierarchical_label'].map(label2id)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=seed, stratify=df['label_id'])

# Tokenizer
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

max_length = 512  # adjust as needed (up to 1024 for GPT-2)

def tokenize_function(batch):
    return tokenizer(
        batch['consumer_complaint_narrative'],
        truncation=True,
        max_length=max_length,
        padding='max_length'
    )

train_ds = Dataset.from_pandas(train_df[['consumer_complaint_narrative', 'label_id']])
val_ds = Dataset.from_pandas(val_df[['consumer_complaint_narrative', 'label_id']])

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)

train_ds = train_ds.rename_column('label_id', 'labels')
val_ds = val_ds.rename_column('label_id', 'labels')

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================================
# 5) Precision settings: bf16 if available, else fp32
# ==========================================================
def gpu_supports_bf16():
    if not torch.cuda.is_available():
        return False
    major, minor = torch.cuda.get_device_capability()
    return major >= 8  # Ampere (A100) or newer

use_bf16 = gpu_supports_bf16()
use_fp16 = False  # Keep fp16 disabled to avoid GradScaler unscale error

print(f"GPU available: {torch.cuda.is_available()}, BF16 support: {use_bf16}, FP16 enabled: {use_fp16}")

# Prefer new TF32 API for speed (optional)
try:
    torch.backends.cuda.matmul.fp32_precision = "tf32"   # 'ieee' or 'tf32'
    torch.backends.cudnn.conv.fp32_precision = "tf32"
except Exception as e:
    pass

# ==========================================================
# 6) Model + LoRA (GPT-2 uses Conv1D; fan_in_fan_out=True)
# ==========================================================
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
       torch_dtype=(torch.bfloat16 if use_bf16 else torch.float32)
)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False  # disable cache for training

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['c_attn', 'c_proj'],  # GPT-2 attention QKV + output projection
    fan_in_fan_out=True                   # IMPORTANT for GPT-2 Conv1D orientation
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# ==========================================================
# 7) Trainer setup (will still print a deprecation warning; OK on 4.44.x)
# ==========================================================
def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds_np = np.argmax(logits, axis=-1)

    # Map IDs to hierarchical strings
    y_true_str = [id2label[int(i)] for i in labels_np]
    y_pred_str = [id2label[int(i)] for i in preds_np]

    # Your hierarchical metrics
    h = hierarchical_metrics(y_true_str, y_pred_str)

    # Also include standard metrics on the flat label IDs
    acc = accuracy_score(labels_np, preds_np)
    macro_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)
    weighted_f1 = f1_score(labels_np, preds_np, average='weighted', zero_division=0)

    return {
        'accuracy': float(acc),
        'macro_precision': float(macro_precision),
        'macro_recall': float(macro_recall),
        'macro_f1': float(macro_f1),
        'weighted_f1': float(weighted_f1),
        **{k: float(v) for k, v in h.items()}
    }

output_dir = '/content/gpt2_lora_cls'
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    weight_decay=0.01,

    evaluation_strategy='epoch',  # deprecation warning OK on 4.44.x
    logging_strategy='steps',
    logging_steps=50,
    save_strategy='epoch',

    load_best_model_at_end=True,
    metric_for_best_model='eval_hierarchical_f1',
    greater_is_better=True,

    remove_unused_columns=False,
    report_to='none',

    fp16=use_fp16,
    bf16=use_bf16,
    tf32=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# ==========================================================
# 8) Train
# ==========================================================
trainer.train()

# ==========================================================
# 9) Evaluate using YOUR hierarchical metrics explicitly
# ==========================================================
eval_preds = trainer.predict(val_ds)
pred_ids = np.argmax(eval_preds.predictions, axis=-1)
true_ids = eval_preds.label_ids

y_true_str = [id2label[int(i)] for i in true_ids]
y_pred_str = [id2label[int(i)] for i in pred_ids]
hier_eval = hierarchical_metrics(y_true_str, y_pred_str)

print("\n=== Hierarchical Evaluation (YOUR functions) ===")
for k, v in hier_eval.items():
    print(f"{k}: {float(v):.4f}")

# Also print a flat classification report over hierarchical label strings
print("\n=== Flat Classification Report (over full hierarchical labels) ===")
print(classification_report(true_ids, pred_ids, target_names=[id2label[i] for i in range(num_labels)], zero_division=0))

# ==========================================================
# 10) Save LoRA adapter + tokenizer, and an inference helper
# ==========================================================
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"\nModel and tokenizer saved to: {output_dir}")

def predict_label(texts):
    if isinstance(texts, str):
        texts = [texts]
    enc = tokenizer(texts, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')
    enc = {k: v.to(model.device) for k, v in enc.items()}
    model.eval()
    with torch.no_grad():
        logits = model(**enc).logits
    preds = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
    return [id2label[p] for p in preds]

# Example:
# sample_text = "My credit card was charged twice and customer service refused to refund."


Num classes (hierarchical_label): 54
Sample classes: ['CheckingSavings::cdCertificateOfDeposit', 'CheckingSavings::checkingAccount', 'CheckingSavings::otherBankingProductOrService', 'CheckingSavings::savingsAccount', 'CreditCard::general-purposeCreditCardOrChargeCard', 'CreditCard::storeCreditCard', 'CreditPrepaidCard::general-purposeCreditCardOrChargeCard', 'CreditPrepaidCard::storeCreditCard', 'CreditReporting::creditReporting', 'CreditReporting::otherPersonalConsumerReport']


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/86400 [00:00<?, ? examples/s]

Map:   0%|          | 0/21600 [00:00<?, ? examples/s]

GPU available: True, BF16 support: True, FP16 enabled: False


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


trainable params: 852,480 || all params: 125,333,760 || trainable%: 0.6801678973007751


Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted F1,Product Precision,Product Recall,Product F1,Sub Product Precision,Sub Product Recall,Sub Product F1,Hierarchical Precision,Hierarchical Recall,Hierarchical F1
1,1.765400,1.790503,0.476852,0.496218,0.476852,0.471406,0.471406,0.717302,0.707037,0.708108,0.507365,0.495972,0.487625,0.612334,0.601505,0.597867
2,1.739500,1.652695,0.516481,0.524509,0.516481,0.508867,0.508867,0.742788,0.733194,0.736005,0.537448,0.532731,0.524057,0.640118,0.632963,0.630031
3,1.740700,1.652534,0.514537,0.525832,0.514537,0.509390,0.509390,0.741076,0.733333,0.735671,0.539414,0.531111,0.525031,0.640245,0.632222,0.630351



=== Hierarchical Evaluation (YOUR functions) ===
product_precision: 0.7411
product_recall: 0.7333
product_f1: 0.7357
sub_product_precision: 0.5394
sub_product_recall: 0.5311
sub_product_f1: 0.5250
hierarchical_precision: 0.6402
hierarchical_recall: 0.6322
hierarchical_f1: 0.6304

=== Flat Classification Report (over full hierarchical labels) ===
                                                            precision    recall  f1-score   support

                   CheckingSavings::cdCertificateOfDeposit       0.80      0.69      0.74       400
                          CheckingSavings::checkingAccount       0.36      0.70      0.48       400
             CheckingSavings::otherBankingProductOrService       0.16      0.08      0.11       400
                           CheckingSavings::savingsAccount       0.60      0.57      0.58       400
         CreditCard::general-purposeCreditCardOrChargeCard       0.44      0.55      0.49       400
                               CreditCard::storeCr

In [4]:

# ================================
# 1) Install known-compatible libs
# ================================
# !pip -q install "transformers==4.44.2" "datasets==2.21.0" "peft==0.10.0" "accelerate==0.34.2" "scikit-learn==1.5.2"

import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

# For reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

print("Torch:", torch.__version__)
import transformers
import peft
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)

# ==========================================
# 2) Load your data (df or a CSV fallback)
# ==========================================
# If you already have df in memory with the two columns, skip this block.
# Otherwise, uncomment and upload a CSV with the two columns: consumer_complaint_narrative, hierarchical_label

# from google.colab import files
# uploaded = files.upload()  # pick your CSV file
# csv_name = list(uploaded.keys())[0]
# df = pd.read_csv(csv_name)

assert 'consumer_complaint_narrative' in df.columns and 'hierarchical_label' in df.columns, \
    "DataFrame must have columns: consumer_complaint_narrative, hierarchical_label"

# Basic cleaning and normalization
df = df[['consumer_complaint_narrative', 'hierarchical_label']].copy()
df.dropna(subset=['consumer_complaint_narrative', 'hierarchical_label'], inplace=True)
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].astype(str).str.strip()
df['hierarchical_label'] = df['hierarchical_label'].astype(str).str.strip()

# Ensure labels are "product::sub-product" format; fill missing sub-product with 'None'
def normalize_hier_label(label: str):
    label = label.strip()
    if '::' in label:
        parts = label.split('::', 1)
        product = parts[0].strip()
        sub = parts[1].strip() if parts[1].strip() else 'None'
        return f"{product}::{sub}"
    else:
        return f"{label}::None"

df['hierarchical_label'] = df['hierarchical_label'].apply(normalize_hier_label)

# ==========================================================
# 3) Your Hierarchical Metric Functions (as provided)
# ==========================================================
def split_hierarchical_label(label):
    if '::' in label:
        return label.split('::')
    else:
        return [label, 'None']

def hierarchical_metrics(y_true, y_pred):
    product_true = [split_hierarchical_label(label)[0] for label in y_true]
    sub_product_true = [split_hierarchical_label(label)[1] for label in y_true]

    product_pred = [split_hierarchical_label(label)[0] for label in y_pred]
    sub_product_pred = [split_hierarchical_label(label)[1] for label in y_pred]

    product_precision = precision_score(product_true, product_pred, average='weighted', zero_division=0)
    product_recall = recall_score(product_true, product_pred, average='weighted', zero_division=0)
    product_f1 = f1_score(product_true, product_pred, average='weighted', zero_division=0)

    valid_sub_product_true = [sub for i, sub in enumerate(sub_product_true) if sub != 'None' and sub_product_pred[i] != 'None']
    valid_sub_product_pred = [sub for i, sub in enumerate(sub_product_pred) if sub != 'None' and sub_product_true[i] != 'None']

    sub_product_precision = precision_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_recall = recall_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_f1 = f1_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0

    hierarchical_precision = (product_precision + sub_product_precision) / 2
    hierarchical_recall = (product_recall + sub_product_recall) / 2
    hierarchical_f1 = (product_f1 + sub_product_f1) / 2

    return {
        'product_precision': product_precision,
        'product_recall': product_recall,
        'product_f1': product_f1,
        'sub_product_precision': sub_product_precision,
        'sub_product_recall': sub_product_recall,
        'sub_product_f1': sub_product_f1,
        'hierarchical_precision': hierarchical_precision,
        'hierarchical_recall': hierarchical_recall,
        'hierarchical_f1': hierarchical_f1
    }

# ==========================================================
# 4) Build label space (full hierarchical strings)
# ==========================================================
labels = sorted(df['hierarchical_label'].unique())
label2id = {lbl: i for i, lbl in enumerate(labels)}
id2label = {i: lbl for lbl, i in label2id.items()}
num_labels = len(labels)

print(f"Num classes (hierarchical_label): {num_labels}")
print("Sample classes:", labels[:10])

# Assign numeric labels
df['label_id'] = df['hierarchical_label'].map(label2id)

# Train/Validation split (stratified)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=seed, stratify=df['label_id'])

# ==========================================================
# 5) Tokenizer and datasets
# ==========================================================
model_name = 'gpt2'  # or 'gpt2-medium' if you have more memory
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT-2 doesn't have a pad token by default; use EOS as padding
tokenizer.pad_token = tokenizer.eos_token

max_length = 512  # up to 1024 for GPT-2; watch memory

def tokenize_function(batch):
    return tokenizer(
        batch['consumer_complaint_narrative'],
        truncation=True,
        max_length=max_length,
        padding='max_length'
    )

train_ds = Dataset.from_pandas(train_df[['consumer_complaint_narrative', 'label_id']])
val_ds = Dataset.from_pandas(val_df[['consumer_complaint_narrative', 'label_id']])

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)

# Rename 'label_id' to 'labels' for Trainer
train_ds = train_ds.rename_column('label_id', 'labels')
val_ds = val_ds.rename_column('label_id', 'labels')

# Set format to PyTorch tensors
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# ==========================================================
# 6) Load GPT-2 for seq classification + LoRA
# ==========================================================
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# Ensure pad token id is set
model.config.pad_token_id = tokenizer.pad_token_id
# Disable cache during training to avoid warnings
model.config.use_cache = False

# LoRA config for GPT-2 (Conv1D-backed linear layers => fan_in_fan_out=True)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['c_attn', 'c_proj'],  # GPT-2 attention QKV + output proj
    fan_in_fan_out=True
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================================
# 7) Metrics (standard + hierarchical)
# ==========================================================
def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds_np = np.argmax(logits, axis=-1)

    # Standard metrics
    acc = accuracy_score(labels_np, preds_np)
    macro_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)
    weighted_f1 = f1_score(labels_np, preds_np, average='weighted', zero_division=0)

    # Hierarchical metrics using your functions
    y_true_str = [id2label[int(i)] for i in labels_np]
    y_pred_str = [id2label[int(i)] for i in preds_np]
    h_metrics = hierarchical_metrics(y_true_str, y_pred_str)

    # Combine (plain Python floats)
    return {
        'accuracy': float(acc),
        'macro_precision': float(macro_precision),
        'macro_recall': float(macro_recall),
        'macro_f1': float(macro_f1),
        'weighted_f1': float(weighted_f1),
        **{k: float(v) for k, v in h_metrics.items()}
    }

# ==========================================================
# 8) Training
# ==========================================================
output_dir = '/content/gpt2_lora_cls'

# Use fp16 on T4/V100; switch to bf16 if A100
use_fp16 = torch.cuda.is_available()
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,   # adjust for memory
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,              # slightly higher LR works well with LoRA
    warmup_ratio=0.1,
    weight_decay=0.01,
    evaluation_strategy='epoch',     # <-- fixed: supported by pinned Transformers
    logging_strategy='steps',
    logging_steps=50,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_hierarchical_f1',  # Trainer will log metrics with 'eval_' prefix
    greater_is_better=True,
    fp16=use_fp16,
    remove_unused_columns=False,      # important for decoder-only models
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

# ==========================================================
# 9) Final Evaluation + Classification Report
# ==========================================================
eval_res = trainer.evaluate()
print("\n=== Final Evaluation (standard + hierarchical) ===")
for k, v in eval_res.items():
    try:
        print(f"{k}: {float(v):.4f}")
    except Exception:
        print(f"{k}: {v}")

# Optional: detailed classification report (flat labels)
preds = trainer.predict(val_ds)
pred_ids = np.argmax(preds.predictions, axis=-1)
true_ids = preds.label_ids

print("\n=== Flat Classification Report (over full hierarchical labels) ===")
print(classification_report(true_ids, pred_ids, target_names=[id2label[i] for i in range(num_labels)], zero_division=0))

# ==========================================================
# 10) Save LoRA adapter + tokenizer
# ==========================================================
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\nModel and tokenizer saved to: {output_dir}")


Torch: 2.9.0+cu126
Transformers: 4.44.2
PEFT: 0.10.0
Num classes (hierarchical_label): 54
Sample classes: ['CheckingSavings::cdCertificateOfDeposit', 'CheckingSavings::checkingAccount', 'CheckingSavings::otherBankingProductOrService', 'CheckingSavings::savingsAccount', 'CreditCard::general-purposeCreditCardOrChargeCard', 'CreditCard::storeCreditCard', 'CreditPrepaidCard::general-purposeCreditCardOrChargeCard', 'CreditPrepaidCard::storeCreditCard', 'CreditReporting::creditReporting', 'CreditReporting::otherPersonalConsumerReport']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/86400 [00:00<?, ? examples/s]

Map:   0%|          | 0/21600 [00:00<?, ? examples/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


trainable params: 852,480 || all params: 125,333,760 || trainable%: 0.6801678973007751


/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


ValueError: Attempting to unscale FP16 gradients.

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
import torch
from transformers import (
    GPT2TokenizerFast,
    GPT2ForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)


In [ ]:
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

# For reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# Basic cleaning and normalization
df = df[['consumer_complaint_narrative', 'hierarchical_label']].copy()
df.dropna(subset=['consumer_complaint_narrative', 'hierarchical_label'], inplace=True)
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].astype(str).str.strip()
df['hierarchical_label'] = df['hierarchical_label'].astype(str).str.strip()


In [ ]:
# Ensure labels are "product::sub-product" format; fill missing sub-product with 'None'
def normalize_hier_label(label: str):
    label = label.strip()
    if '::' in label:
        parts = label.split('::', 1)
        product = parts[0].strip()
        sub = parts[1].strip() if parts[1].strip() else 'None'
        return f"{product}::{sub}"
    else:
        return f"{label}::None"

df['hierarchical_label'] = df['hierarchical_label'].apply(normalize_hier_label)


In [ ]:
# ==========================================================
# 3) Your Hierarchical Metric Functions (as provided)
# ==========================================================
def split_hierarchical_label(label):
    if '::' in label:
        return label.split('::')
    else:
        return [label, 'None']

def hierarchical_metrics(y_true, y_pred):
    product_true = [split_hierarchical_label(label)[0] for label in y_true]
    sub_product_true = [split_hierarchical_label(label)[1] for label in y_true]

    product_pred = [split_hierarchical_label(label)[0] for label in y_pred]
    sub_product_pred = [split_hierarchical_label(label)[1] for label in y_pred]

    product_precision = precision_score(product_true, product_pred, average='weighted', zero_division=0)
    product_recall = recall_score(product_true, product_pred, average='weighted', zero_division=0)
    product_f1 = f1_score(product_true, product_pred, average='weighted', zero_division=0)

    valid_sub_product_true = [sub for i, sub in enumerate(sub_product_true) if sub != 'None' and sub_product_pred[i] != 'None']
    valid_sub_product_pred = [sub for i, sub in enumerate(sub_product_pred) if sub != 'None' and sub_product_true[i] != 'None']

    sub_product_precision = precision_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_recall = recall_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_f1 = f1_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0

    hierarchical_precision = (product_precision + sub_product_precision) / 2
    hierarchical_recall = (product_recall + sub_product_recall) / 2
    hierarchical_f1 = (product_f1 + sub_product_f1) / 2

    return {
        'product_precision': product_precision,
        'product_recall': product_recall,
        'product_f1': product_f1,
        'sub_product_precision': sub_product_precision,
        'sub_product_recall': sub_product_recall,
        'sub_product_f1': sub_product_f1,
        'hierarchical_precision': hierarchical_precision,
        'hierarchical_recall': hierarchical_recall,
        'hierarchical_f1': hierarchical_f1
    }


In [ ]:
# ==========================================================
# 4) Build label space (full hierarchical strings)
# ==========================================================
labels = sorted(df['hierarchical_label'].unique())
label2id = {lbl: i for i, lbl in enumerate(labels)}
id2label = {i: lbl for lbl, i in label2id.items()}
num_labels = len(labels)

print(f"Num classes (hierarchical_label): {num_labels}")
print("Sample classes:", labels[:10])

# Assign numeric labels
df['label_id'] = df['hierarchical_label'].map(label2id)

# Train/Validation split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=seed, stratify=df['label_id'])

# ==========================================================
# 5) Tokenizer and datasets
# ==========================================================
model_name = 'gpt2'  # you can use 'gpt2-medium' if you have more GPU memory
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

# GPT-2 doesn't have a pad token by default; use EOS as padding
tokenizer.pad_token = tokenizer.eos_token

max_length = 512  # adjust as needed (GPT-2 supports up to 1024)

def tokenize_function(batch):
    return tokenizer(
        batch['consumer_complaint_narrative'],
        truncation=True,
        max_length=max_length,
        padding='max_length'
    )

train_ds = Dataset.from_pandas(train_df[['consumer_complaint_narrative', 'label_id']])
val_ds = Dataset.from_pandas(val_df[['consumer_complaint_narrative', 'label_id']])

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)

# Rename 'label_id' to 'labels' for Trainer
train_ds = train_ds.rename_column('label_id', 'labels')
val_ds = val_ds.rename_column('label_id', 'labels')

# Set format to PyTorch tensors
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# ==========================================================
# 6) Load GPT-2 for sequence classification + LoRA
# ==========================================================
model = GPT2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

# Ensure pad token id is set
model.config.pad_token_id = tokenizer.pad_token_id

# LoRA config for GPT-2
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['c_attn', 'c_proj']  # common GPT-2 linear layers for attention
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================================
# 7) Metrics (standard + hierarchical)
# ==========================================================
def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds_np = np.argmax(logits, axis=-1)

    # Standard metrics
    acc = accuracy_score(labels_np, preds_np)
    macro_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)
    weighted_f1 = f1_score(labels_np, preds_np, average='weighted', zero_division=0)

    # Hierarchical metrics using your functions
    y_true_str = [id2label[int(i)] for i in labels_np]
    y_pred_str = [id2label[int(i)] for i in preds_np]
    h_metrics = hierarchical_metrics(y_true_str, y_pred_str)

    # Combine
    return {
        'accuracy': acc,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1,
        **h_metrics
    }

# ==========================================================
# 8) Training
# ==========================================================
output_dir = '/content/gpt2_lora_cls'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,   # adjust based on GPU memory
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,              # LoRA often allows slightly higher LR
    warmup_ratio=0.1,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_strategy='steps',
    logging_steps=50,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_hierarchical_f1',  # Select best on hierarchical F1
    greater_is_better=True,
    fp16=True,                        # Colab GPU supports mixed precision
    report_to='none'                  # disable W&B etc.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

# ==========================================================
# 9) Final Evaluation + Classification Report
# ==========================================================
eval_res = trainer.evaluate()
print("\n=== Final Evaluation (standard + hierarchical) ===")
for k, v in eval_res.items():
    if isinstance(v, (float, np.floating)):
        print(f"{k}: {v:.4f}")
    else:
        print(f"{k}: {v}")

# Optional: detailed classification report (flat labels)
preds = trainer.predict(val_ds)
pred_ids = np.argmax(preds.predictions, axis=-1)
true_ids = preds.label_ids

print("\n=== Flat Classification Report (weighted/macro over full hierarchical labels) ===")
print(classification_report(true_ids, pred_ids, target_names=[id2label[i] for i in range(num_labels)], zero_division=0))

# ==========================================================
# 10) Save LoRA adapter + tokenizer
# ==========================================================
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\nModel and tokenizer saved to: {output_dir}")

# ==========================================================
# 11) Inference helper
# ==========================================================
def predict_label(texts):
    if isinstance(texts, str):
        texts = [texts]
    enc = tokenizer(texts, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')
    enc = {k: v.to(model.device) for k, v in enc.items()}
    with torch.no_grad():
        logits = model(**enc).logits
    preds = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
    return [id2label[p] for p in preds]

# Example:
# sample_text = "My credit card was charged twice and customer service refused to refund."
print(predict_label(sample_text))

Num classes (hierarchical_label): 54
Sample classes: ['CheckingSavings::cdCertificateOfDeposit', 'CheckingSavings::checkingAccount', 'CheckingSavings::otherBankingProductOrService', 'CheckingSavings::savingsAccount', 'CreditCard::general-purposeCreditCardOrChargeCard', 'CreditCard::storeCreditCard', 'CreditPrepaidCard::general-purposeCreditCardOrChargeCard', 'CreditPrepaidCard::storeCreditCard', 'CreditReporting::creditReporting', 'CreditReporting::otherPersonalConsumerReport']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 852,480 || all params: 125,333,760 || trainable%: 0.6802


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:

# ================================
# 1) Install known-compatible libs
# ================================
# !pip -q install "transformers==4.44.2" "datasets==2.21.0" "peft==0.10.0" "accelerate==0.34.2" "scikit-learn==1.5.2"

import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

# For reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

print("Torch:", torch.__version__)
import transformers
import peft
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)

# ==========================================
# 2) Load your data (df or a CSV fallback)
# ==========================================
# If you already have df in memory with the two columns, skip this block.
# Otherwise, uncomment and upload a CSV with the two columns: consumer_complaint_narrative, hierarchical_label

# from google.colab import files
# uploaded = files.upload()  # pick your CSV file
# csv_name = list(uploaded.keys())[0]
# df = pd.read_csv(csv_name)

assert 'consumer_complaint_narrative' in df.columns and 'hierarchical_label' in df.columns, \
    "DataFrame must have columns: consumer_complaint_narrative, hierarchical_label"

# Basic cleaning and normalization
df = df[['consumer_complaint_narrative', 'hierarchical_label']].copy()
df.dropna(subset=['consumer_complaint_narrative', 'hierarchical_label'], inplace=True)
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].astype(str).str.strip()
df['hierarchical_label'] = df['hierarchical_label'].astype(str).str.strip()

# Ensure labels are "product::sub-product" format; fill missing sub-product with 'None'
def normalize_hier_label(label: str):
    label = label.strip()
    if '::' in label:
        parts = label.split('::', 1)
        product = parts[0].strip()
        sub = parts[1].strip() if parts[1].strip() else 'None'
        return f"{product}::{sub}"
    else:
        return f"{label}::None"

df['hierarchical_label'] = df['hierarchical_label'].apply(normalize_hier_label)

# ==========================================================
# 3) Your Hierarchical Metric Functions (as provided)
# ==========================================================
def split_hierarchical_label(label):
    if '::' in label:
        return label.split('::')
    else:
        return [label, 'None']

def hierarchical_metrics(y_true, y_pred):
    product_true = [split_hierarchical_label(label)[0] for label in y_true]
    sub_product_true = [split_hierarchical_label(label)[1] for label in y_true]

    product_pred = [split_hierarchical_label(label)[0] for label in y_pred]
    sub_product_pred = [split_hierarchical_label(label)[1] for label in y_pred]

    product_precision = precision_score(product_true, product_pred, average='weighted', zero_division=0)
    product_recall = recall_score(product_true, product_pred, average='weighted', zero_division=0)
    product_f1 = f1_score(product_true, product_pred, average='weighted', zero_division=0)

    valid_sub_product_true = [sub for i, sub in enumerate(sub_product_true) if sub != 'None' and sub_product_pred[i] != 'None']
    valid_sub_product_pred = [sub for i, sub in enumerate(sub_product_pred) if sub != 'None' and sub_product_true[i] != 'None']

    sub_product_precision = precision_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_recall = recall_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_f1 = f1_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0

    hierarchical_precision = (product_precision + sub_product_precision) / 2
    hierarchical_recall = (product_recall + sub_product_recall) / 2
    hierarchical_f1 = (product_f1 + sub_product_f1) / 2

    return {
        'product_precision': product_precision,
        'product_recall': product_recall,
        'product_f1': product_f1,
        'sub_product_precision': sub_product_precision,
        'sub_product_recall': sub_product_recall,
        'sub_product_f1': sub_product_f1,
        'hierarchical_precision': hierarchical_precision,
        'hierarchical_recall': hierarchical_recall,
        'hierarchical_f1': hierarchical_f1
    }

# ==========================================================
# 4) Build label space (full hierarchical strings)
# ==========================================================
labels = sorted(df['hierarchical_label'].unique())
label2id = {lbl: i for i, lbl in enumerate(labels)}
id2label = {i: lbl for lbl, i in label2id.items()}
num_labels = len(labels)

print(f"Num classes (hierarchical_label): {num_labels}")
print("Sample classes:", labels[:10])

# Assign numeric labels
df['label_id'] = df['hierarchical_label'].map(label2id)

# Train/Validation split (stratified)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=seed, stratify=df['label_id'])

# ==========================================================
# 5) Tokenizer and datasets
# ==========================================================
model_name = 'gpt2'  # or 'gpt2-medium' if you have more memory
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT-2 doesn't have a pad token by default; use EOS as padding
tokenizer.pad_token = tokenizer.eos_token

max_length = 512  # up to 1024 for GPT-2; watch memory

def tokenize_function(batch):
    return tokenizer(
        batch['consumer_complaint_narrative'],
        truncation=True,
        max_length=max_length,
        padding='max_length'
    )

train_ds = Dataset.from_pandas(train_df[['consumer_complaint_narrative', 'label_id']])
val_ds = Dataset.from_pandas(val_df[['consumer_complaint_narrative', 'label_id']])

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)

# Rename 'label_id' to 'labels' for Trainer
train_ds = train_ds.rename_column('label_id', 'labels')
val_ds = val_ds.rename_column('label_id', 'labels')

# Set format to PyTorch tensors
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# ==========================================================
# 6) Load GPT-2 for seq classification + LoRA
# ==========================================================
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

# Ensure pad token id is set
model.config.pad_token_id = tokenizer.pad_token_id
# Disable cache during training to avoid warnings
model.config.use_cache = False

# LoRA config for GPT-2 (Conv1D-backed linear layers => fan_in_fan_out=True)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['c_attn', 'c_proj'],  # GPT-2 attention QKV + output proj
    fan_in_fan_out=True
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================================
# 7) Metrics (standard + hierarchical)
# ==========================================================
def compute_metrics(eval_pred):
    logits, labels_np = eval_pred
    preds_np = np.argmax(logits, axis=-1)

    # Standard metrics
    acc = accuracy_score(labels_np, preds_np)
    macro_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
    macro_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)
    weighted_f1 = f1_score(labels_np, preds_np, average='weighted', zero_division=0)

    # Hierarchical metrics using your functions
    y_true_str = [id2label[int(i)] for i in labels_np]
    y_pred_str = [id2label[int(i)] for i in preds_np]
    h_metrics = hierarchical_metrics(y_true_str, y_pred_str)

    # Combine (plain Python floats)
    return {
        'accuracy': float(acc),
        'macro_precision': float(macro_precision),
        'macro_recall': float(macro_recall),
        'macro_f1': float(macro_f1),
        'weighted_f1': float(weighted_f1),
        **{k: float(v) for k, v in h_metrics.items()}
    }

# ==========================================================
# 8) Training
# ==========================================================
output_dir = '/content/gpt2_lora_cls'

# Use fp16 on T4/V100; switch to bf16 if A100
use_fp16 = torch.cuda.is_available()
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,   # adjust for memory
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,              # slightly higher LR works well with LoRA
    warmup_ratio=0.1,
    weight_decay=0.01,
    evaluation_strategy='epoch',     # <-- fixed: supported by pinned Transformers
    logging_strategy='steps',
    logging_steps=50,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_hierarchical_f1',  # Trainer will log metrics with 'eval_' prefix
    greater_is_better=True,
    fp16=use_fp16,
    remove_unused_columns=False,      # important for decoder-only models
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

# ==========================================================
# 9) Final Evaluation + Classification Report
# ==========================================================
eval_res = trainer.evaluate()
print("\n=== Final Evaluation (standard + hierarchical) ===")
for k, v in eval_res.items():
    try:
        print(f"{k}: {float(v):.4f}")
    except Exception:
        print(f"{k}: {v}")

# Optional: detailed classification report (flat labels)
preds = trainer.predict(val_ds)
pred_ids = np.argmax(preds.predictions, axis=-1)
true_ids = preds.label_ids

print("\n=== Flat Classification Report (over full hierarchical labels) ===")
print(classification_report(true_ids, pred_ids, target_names=[id2label[i] for i in range(num_labels)], zero_division=0))

# ==========================================================
# 10) Save LoRA adapter + tokenizer
# ==========================================================
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\nModel and tokenizer saved to: {output_dir}")


Torch: 2.9.0+cu126
Transformers: 4.57.3
PEFT: 0.18.0
Num classes (hierarchical_label): 54
Sample classes: ['CheckingSavings::cdCertificateOfDeposit', 'CheckingSavings::checkingAccount', 'CheckingSavings::otherBankingProductOrService', 'CheckingSavings::savingsAccount', 'CreditCard::general-purposeCreditCardOrChargeCard', 'CreditCard::storeCreditCard', 'CreditPrepaidCard::general-purposeCreditCardOrChargeCard', 'CreditPrepaidCard::storeCreditCard', 'CreditReporting::creditReporting', 'CreditReporting::otherPersonalConsumerReport']


Map:   0%|          | 0/86400 [00:00<?, ? examples/s]

Map:   0%|          | 0/21600 [00:00<?, ? examples/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 852,480 || all params: 125,333,760 || trainable%: 0.6802


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# ==========================================================
# 11) Inference helper
# ==========================================================
def predict_label(texts):
    if isinstance(texts, str):
        texts = [texts]
    enc = tokenizer(texts, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')
    enc = {k: v.to(model.device) for k, v in enc.items()}
    model.eval()
    with torch.no_grad():
      logits = model(**enc).logits
    preds = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
    return [id2label[p] for p in preds]
# Example:
# sample_text = "My credit card was charged twice and customer service refused to refund."
# Example:
# sample_text = "My credit card was charged twice and customer service refused to refund."
print(predict_label(sample_text))